## SRJ

## Using ROHF

In [ ]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged) 
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.ROHF(mol_pyscf)
rhf.kernel()

converged SCF energy = -1655.34968383623


np.float64(-1655.3496838362307)

In [ ]:
one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc') #Kinetic energy integrals + Nuclear energy integrals = Hcore
two_ao = mol_pyscf.intor('int2e_sph') # Two electron integrals

print(rhf.mo_coeff.shape)  # Should be (num_AOs, num_MOs)
print(one_ao.shape)  
print(two_ao.shape)

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)


two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)

core_constant = np.array([rhf.energy_nuc()])

#H_fermionic_full = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

#H_full = qml.jordan_wigner(H_fermionic_full)

(27, 27)
(27, 27)
(27, 27, 27, 27)


In [2]:
import pennylane as qml 
from pennylane import qchem


one_ao = mol_pyscf.intor_symmetric("int1e_kin") + mol_pyscf.intor_symmetric("int1e_nuc")
two_ao = mol_pyscf.intor("int2e_sph")


one_mo = np.einsum("pi,pq,qj->ij", rhf.mo_coeff, one_ao, rhf.mo_coeff, optimize=True)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

core_constant = np.array([rhf.energy_nuc()])
print('The value of core:', core_constant)

# convert the two-electron integral tensor to the physicists’ notation
two_mo = np.swapaxes(two_mo, 1, 3)


core=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17] 
active=[18, 19, 20, 21, 22, 23]

total_orb = len(core) + len(active)
print('Total orbitals are',total_orb)

# define the active space and recompute the integrals
def pyscf_integral(core, active, one_mo, two_mo, core_constant):
    if core and active:
        for i in core:
            core_constant = core_constant + 2 * one_mo[i][i]
            for j in core:
                core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]

        for p in active:
            for q in active:
                for i in core:
                    print(f'The value of p and q', {p}, {q})
                    one_mo[p, q] = one_mo[p, q] + (2 * two_mo[i][p][q][i] - two_mo[i][p][i][q])

        one_mo = one_mo[qml.math.ix_(active, active)]
        two_mo = two_mo[qml.math.ix_(active, active, active, active)]

    return core_constant, one_mo, two_mo

core_constant, one_mo, two_mo = pyscf_integral(core, active, one_mo,two_mo, core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)
print(H)

The value of core: [120.55734926]
Total orbitals are 24
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p

# Old form of converting full space to active space

active_indices = [18, 19, 20, 21, 22, 23]

one_mo_active = one_mo[np.ix_(active_indices, active_indices)]

import numpy as np

two_mo_active = two_mo[np.ix_(active_indices, active_indices, active_indices, active_indices)]

core_constant_active = core_constant  # Usually unchanged for active space

import pennylane as qml

H_fermionic_active = qml.qchem.fermionic_observable(core_constant_active, one_mo_active, two_mo_active)
H_active = qml.jordan_wigner(H_fermionic_active)


## Using hf_state similar to qiskit

In [ ]:




#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
#warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(active_electrons, H, adapt_it):
    print('Using active space, check if you change the H accordingly')
    qubits = 12
    active_electrons = active_electrons
    #print(H)
    #hf_state = qchem.hf_state(active_electrons, qubits)
    hf_state = np.array([1,1,1,0,0,0,1,1,1,0,0,0])
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad






#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 6


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact( active_electrons, H, adapt_it=50) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


Using active space, check if you change the H accordingly
HF state is -1655.1484203124342
Total excitations are 117
The adapt iteration now is 0


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.10812497583517312 for operator a⁺(0) a⁺(1) a(10) a(11)
Highest gradient excitation is [0, 1, 10, 11]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [0.25584954]
Final cost: -1655.1553689998254
 0: ──X─╭FermionicDoubleExcitation(0.26)─┤  State
 1: ──X─├FermionicDoubleExcitation(0.26)─┤  State
 2: ──X─│────────────────────────────────┤  State
 6: ──X─│────────────────────────────────┤  State
 7: ──X─│────────────────────────────────┤  State
 8: ──X─│────────────────────────────────┤  State
10: ────├FermionicDoubleExcitation(0.26)─┤  State
11: ────╰FermionicDoubleExcitation(0.26)─┤  State
The adapt iteration now is 1


...

Final updated parameters: [ 0.26304611 -2.5982032   0.2510491  -0.08492424  0.02567122  0.00877384]
Final cost: -1655.26739356342
 0: ──X─╭FermionicDoubleExcitation(0.26)────────────────────────────────── ···
 1: ──X─├FermionicDoubleExcitation(0.26)────────────────────────────────── ···
 2: ──X─│───────────────────────────────────────────────────────────────── ···
 3: ────│───────────────────────────────────────────────────────────────── ···
 4: ────│────────────────────────────────╭FermionicDoubleExcitation(-2.60) ···
 5: ────│────────────────────────────────├FermionicDoubleExcitation(-2.60) ···
 6: ──X─│────────────────────────────────├FermionicDoubleExcitation(-2.60) ···
 7: ──X─│────────────────────────────────╰FermionicDoubleExcitation(-2.60) ···
 8: ──X─│───────────────────────────────────────────────────────────────── ···
 9: ────│───────────────────────────────────────────────────────────────── ···
10: ────├FermionicDoubleExcitation(0.26)────────────────────────────────── ···
1

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.004008884550335012 for operator a⁺(3) a⁺(4) a(7) a(8)
Highest gradient excitation is [3, 4, 7, 8]
Final updated parameters: [ 0.26505979 -1.58520299  0.72231425 -0.09773194  0.01967443  0.0076606
 -2.40582342]
Final cost: -1655.2741233804677
 0: ──X─╭FermionicDoubleExcitation(0.27)────────────────────────────────── ···
 1: ──X─├FermionicDoubleExcitation(0.27)────────────────────────────────── ···
 2: ──X─│───────────────────────────────────────────────────────────────── ···
 3: ────│───────────────────────────────────────────────────────────────── ···
 4: ────│────────────────────────────────╭FermionicDoubleExcitation(-1.59) ···
 5: ────│────────────────────────────────├FermionicDoubleExcitation(-1.59) ···
 6: ──X─│────────────────────────────────├FermionicDoubleExcitation(-1.59) ···
 7: ──X─│────────────────────────────────╰FermionicDoubleExcitation(-1.59) ···
 8: ──X─│───────────────────────────────────────────────────────────────── ···
 9: ────│──────

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.0030753963056279155 for operator a⁺(4) a(10)
Highest gradient excitation is [4, 10]
